### Imports

In [3]:
import face_recognition
import cv2
import numpy as np
import serial
from serial import Serial
from PIL import Image
import time
import os
from os import listdir
from os.path import isfile, join, exists

print("All libraries are installed successfully!")

All libraries are installed successfully!


### Testing for Available Cameras

In [3]:
def find_available_cameras(max_index=10):
    """Test all camera indices up to max_index and return a list of available camera indices."""
    available_cameras = []
    for i in range(3):
        video_capture = cv2.VideoCapture(i)
        if video_capture.isOpened():
            print(f"Camera {i} is available.")
            available_cameras.append(i)
            video_capture.release()  # Release the camera after checking
        else:
            print(f"Camera {i} is not available.")
    return available_cameras

# Find all available cameras
available_cameras = find_available_cameras()

if available_cameras:
    print(f"Available cameras: {available_cameras}")
else:
    print("No cameras found.")

2024-12-11 12:59:03.096 python[48781:2526464] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Camera 0 is available.
Camera 1 is available.
Camera 2 is not available.
Available cameras: [0, 1]


OpenCV: out device of bound (0-1): 2
OpenCV: camera failed to properly initialize!


In [ ]:
def list_available_cameras():
    """List all available camera indices."""
    available_cameras = []
    for i in range(5):  # Test first 5 indices
        video_capture = cv2.VideoCapture(i)
        if video_capture.isOpened():
            available_cameras.append(i)
            video_capture.release()  # Release the camera after testing
    return available_cameras

# Get the list of available cameras
available_cameras = list_available_cameras()

if not available_cameras:
    print("No working cameras found.")
else:
    print("Available cameras:", available_cameras)
    # You can also use any available camera by selecting an index from the list:
    for cam in available_cameras:
        print(f"Using camera index: {cam}")
        video_capture = cv2.VideoCapture(cam)

        start_time = time.time()  # Get the current time

        while True:
            ret, frame = video_capture.read()
            if not ret:
                print("Failed to capture frame. Exiting...")
                break

            # Display the frame
            cv2.imshow(f"Camera {cam}", frame)

            # Check if 5 seconds have passed
            if time.time() - start_time > 5:
                print(f"Stopping camera {cam} after 5 seconds.")
                break

            # Allow 'q' key to quit early
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        video_capture.release()
        cv2.destroyAllWindows()

### Model Training

In [6]:
import pickle

# Path to the folders
base_folder = "converted"
folders = ["Abigail", "Audrey"]

# Initialize known face encodings and names
known_face_encodings = []
known_face_names = []

# Train the model with all images in the specified folders
for folder in folders:
    folder_path = os.path.join(base_folder, folder)
    person_name = folder  # Use the folder name as the person's name

    # Loop through each image in the folder
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)

        # Load the image and extract face encodings
        image = face_recognition.load_image_file(image_path)
        face_encodings = face_recognition.face_encodings(image)

        # If at least one face is detected, use the first encoding
        if face_encodings:
            known_face_encodings.append(face_encodings[0])
            known_face_names.append(person_name)

# Print a summary
print(f"Trained on {len(known_face_encodings)} face(s) from {len(folders)} folder(s).")

# After training the model
model_data = {
    "encodings": known_face_encodings,
    "names": known_face_names
}

# Save the model to a file
with open("face_recognition_model.pkl", "wb") as model_file:
    pickle.dump(model_data, model_file)

print("Model saved to face_recognition_model.pkl")

Trained on 79 face(s) from 2 folder(s).
Model saved to face_recognition_model.pkl


### Communication Testing (reading signals from arduino)

In [ ]:
import serial
ser = serial.Serial('/dev/cu.usbmodem1201', 9600)
while True:
    print(ser.readline().decode('utf-8', errors='ignore').strip())

### Main Face Recognition Program Code

In [1]:
import logging
import time
import cv2
import face_recognition
import serial
import pickle
import numpy as np

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Initialize serial communication
try:
    ser = serial.Serial('/dev/cu.usbmodem1201', 9600)
    ser.flushInput()  # Clear serial input buffer to avoid residual noise
    time.sleep(2)  # Allow time for the Arduino to reset
    logging.info("Serial communication initialized.")
except serial.SerialException as e:
    logging.error(f"Serial communication error: {e}")
    exit(1)

# Error-checking function for serial messages
def validate_message(message):
    if not message or len(message) < 2:  # Basic length check
        return False
    allowed_messages = ["FINGERPRINT_OK", "RETRY", "READY_TO_SCAN"]
    return message in allowed_messages or "Communication error" not in message

# Load known face encodings and names
try:
    with open("face_recognition_model.pkl", "rb") as model_file:
        model_data = pickle.load(model_file)
        known_face_encodings = model_data["encodings"]
        known_face_names = model_data["names"]
    logging.info(f"Loaded model with {len(known_face_encodings)} known face(s).")
except FileNotFoundError:
    logging.error("Model file not found. Ensure 'face_recognition_model.pkl' exists.")
    known_face_encodings = []
    known_face_names = []

# Preload lock/unlock images
lock_image = cv2.imread('locked.png')
unlock_image = cv2.imread('rllyLocked.jpg')

if lock_image is None or unlock_image is None:
    logging.error("Could not load lock/unlock status images.")
    lock_image = np.zeros((200, 200, 3), dtype=np.uint8)
    unlock_image = np.full((200, 200, 3), (0, 255, 0), dtype=np.uint8)

# Initialize webcam
video_capture = cv2.VideoCapture(0)

if not video_capture.isOpened():
    logging.error("Camera failed to initialize.")
    exit(1)

# Main loop
while True:
    # Wait for fingerprint verification
    logging.info("Waiting for fingerprint verification...")
    while True:
        if ser.in_waiting > 0:
            raw_message = ser.readline()
            message = raw_message.decode('utf-8', errors='ignore').strip()

            if not validate_message(message):
                logging.warning(f"Invalid or noisy message received: {message}")
                continue  # Skip to the next iteration

            logging.info(f"Received message: {message}")

            if "FINGERPRINT_OK" in message:
                logging.info("Fingerprint verified. Starting face recognition.")
                break
            elif message == "RETRY":
                logging.warning("Fingerprint retry requested.")
            elif message == "READY_TO_SCAN":
                logging.info("Ready to scan fingerprint.")
            else:
                logging.warning(f"Unknown message received: {message}")
        time.sleep(0.1)

    # Start face recognition
    logging.info("Starting face recognition...")
    recognition_threshold = 0.4
    face_detected_start = None
    unlocked = False
    start_time = time.time()

    while not unlocked:
        ret, frame = video_capture.read()
        if not ret:
            logging.error("Failed to capture frame.")
            break

        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        rgb_small_frame = small_frame[:, :, ::-1]

        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            if len(known_face_encodings) == 0:
                logging.warning("No known faces are loaded. Skipping recognition.")
                name = "Unknown"
            else:
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)

                if face_distances[best_match_index] < recognition_threshold:
                    name = known_face_names[best_match_index]
                else:
                    name = "Unknown"

            face_names.append(name)

        if "Abigail" in face_names:  # Replace with desired name
            if face_detected_start is None:
                face_detected_start = time.time()
            elif time.time() - face_detected_start >= 5:  # Detected for 5 seconds
                unlocked = True
                ser.write(b"FACE_RECOGNIZED\n")  # Notify Arduino of success
                logging.info("Face recognized: ABIGAIL. Unlocking door.")
                ser.write(b"DISPLAY:Unlocked\n")
                cv2.imshow('Lock Status', unlock_image)
                cv2.waitKey(3000)  # Display unlocked status
                break
        else:
            # ser.write(b"FACE_NOT_RECOGNIZED\n")  # Notify Arduino of failure
            face_detected_start = None

        if time.time() - start_time > 40:  # Timeout after 40 seconds
            logging.warning("Timeout: No face detected within 40 seconds.")
            ser.write(b"FACE_NOT_RECOGNIZED\n")  # Notify Arduino of timeout
            break

        for (top, right, bottom, left), name in zip(face_locations, face_names):
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        cv2.imshow('Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    if not unlocked:
        ser.write(b"LOCK\n")
        logging.info("Locking door due to timeout.")
        ser.write(b"DISPLAY:Locked\n")
        cv2.imshow('Lock Status', lock_image)
        cv2.waitKey(3000)  # Display locked status

    # Restart process by continuing the main loop
    logging.info("Resetting to fingerprint verification state...")


Dlib was compiled to use AVX instructions, but these aren't available on your machine.
2024-12-16 01:10:50,125 - INFO - Serial communication initialized.
2024-12-16 01:10:50,134 - INFO - Loaded model with 79 known face(s).
2024-12-16 01:10:52.630 python[97063:1642776] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2024-12-16 01:10:54,941 - INFO - Waiting for fingerprint verification...
2024-12-16 01:10:54,943 - INFO - Received message: READY_TO_SCAN
2024-12-16 01:10:54,943 - INFO - Ready to scan fingerprint.
2024-12-16 01:10:58,638 - INFO - Received message: .......RETRY
2024-12-16 01:10:58,638 - WARNING - Unknown message received: .......RETRY
2024-12-16 01:11:04,574 - INFO - Received message: .......RETRY
2024-12-16 01:11:04,575 - WARNING - Unknown message received: .......RETRY
2024-12-16 01:11:09,987 - INFO - Received message: .......RETRY
202

OSError: [Errno 6] Device not configured